In [24]:
import pandas as pd
from datetime import datetime
import pendulum
import pytz
%matplotlib inline

In [25]:
pdf = pd.read_csv('training_data/utah_positive_examples.csv')
ndf = pd.read_csv('training_data/utah_negative_examples.csv')
wdf = pd.read_csv('utah_weather_2010-2018_grouped.csv')

In [26]:
ndf.head()

,timestamp,segment_id,station_id,hour,weekday,month,pre_dir,street_type,suf_dir,one_way,...,aadt,sinuosity,euclidean_length,segment_length,near_major_road,road_orient_approx,at_intersection,near_billboard,accident_counts,target
0,2014-12-22 19:00:00,385152,9999993081,17,0,12,,,,0,...,0.0,1.020150,639.724684,652.615291,0,1.681712,0,0,0.0,0
1,2017-12-06 09:00:00,5753,72572024127,9,2,12,S,,E,0,...,11763.0,1.000000,74.051639,74.051639,0,1.562353,0,0,3.0,0
2,2013-07-01 12:00:00,324415,72472393025,8,0,7,,,,0,...,0.0,1.003237,96.842309,97.155800,0,2.961512,0,0,0.0,0
3,2013-07-06 08:00:00,153026,72572424174,8,5,7,W,,S,0,...,0.0,1.000000,197.361685,197.361685,0,0.018032,1,0,28.0,0
4,2010-03-01 09:00:00,269209,74003024103,16,0,3,,RD,,0,...,0.0,1.128169,1965.866308,2217.828527,0,0.701536,0,0,0.0,0


In [27]:
utah_tz = pendulum.timezone('US/Mountain')

pdf['timestamp'] = pd.to_datetime(pdf.timestamp).apply(lambda x: utah_tz.convert(x.to_pydatetime()))
ndf['timestamp'] = pd.to_datetime(ndf.timestamp).apply(lambda x: utah_tz.convert(x.to_pydatetime()))
wdf['timestamp'] = pd.to_datetime(wdf.timestamp).apply(pytz.utc.localize)

In [28]:
wdf.head()

,station_id,timestamp,snowing,raining,foggy,icy,hailing,thunderstorm,wind_speed,visibility,temperature,precip_depth,snow_depth
0,72470093141,2010-01-01 00:00:00+00:00,False,False,False,False,False,False,2.1,16093.0,-11.7,0.0,0.0
1,72470093141,2010-01-01 01:00:00+00:00,False,False,False,False,False,False,2.6,16093.0,-11.7,0.0,0.0
2,72470093141,2010-01-01 02:00:00+00:00,False,False,False,False,False,False,2.1,16093.0,-12.2,0.0,0.0
3,72470093141,2010-01-01 03:00:00+00:00,False,False,False,False,False,False,2.6,16093.0,-11.7,0.0,0.0
4,72470093141,2010-01-01 04:00:00+00:00,False,False,False,False,False,False,2.1,16093.0,-9.4,0.0,0.0


In [29]:
wdf['timestamp'] = pd.DatetimeIndex(wdf.timestamp).tz_convert('US/Mountain')

In [30]:
def add_join_key(df):
    df['join_key'] = df.station_id.map(int).map(str)+df.timestamp.map(datetime.isoformat)
    df = df.set_index('join_key')
    return df

In [31]:
pdf = add_join_key(pdf)
ndf = add_join_key(ndf)
wdf = add_join_key(wdf)

In [32]:
pdf = pdf.join(wdf.drop(columns=['timestamp', 'station_id']))
ndf = ndf.join(wdf.drop(columns=['timestamp', 'station_id']))

In [33]:
tset = pdf.append(ndf)

In [34]:
tset.to_csv('training_data/utah_training_set.csv')